In [9]:
import os
import cv2
import shutil
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from collections import defaultdict
from ultralytics import YOLO
from scipy import stats
from tqdm import tqdm
from PIL import Image

# Preprocesamiento

In [10]:
# Metadata para ordenar el entrenamiento
metadata_path = 'datasets\Labeled Stomatal Images.csv'
metadata = pd.read_csv(metadata_path)

In [11]:
# Mediciones 1 plant-out
def onePlantOut(plant_name):
  main_folder = 'datasets'

  train_images_folder = os.path.join(main_folder, 'train/images')
  train_labels_folder = os.path.join(main_folder, 'train/labels')
  val_images_folder = os.path.join(main_folder, 'val/images')
  val_labels_folder = os.path.join(main_folder, 'val/labels')

  folders = [train_images_folder, train_labels_folder, val_images_folder, val_labels_folder]

  # Sino existen los directorios
  for folder in folders:
      if not os.path.exists(folder):
          os.makedirs(folder)

  # Seleccionar la planta a medir su rendimiento
  plant = plant_name
  trainFiles = metadata[metadata['Species'] != plant]['FileName'].tolist()
  valFiles = metadata[metadata['Species'] == plant]['FileName'].tolist()

  for folder in folders:
    for file in os.listdir(folder):
      if file.split(".")[0] in trainFiles:
        if file.endswith(".jpg"):
          shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'train/images', file))
        elif file.endswith(".txt"):
          shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'train/labels', file))

  for folder in folders:
    for file in os.listdir(folder):
      if file.split(".")[0] in valFiles:
        if file.endswith(".jpg"):
          shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'val/images', file))
        elif file.endswith(".txt"):
          shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'val/labels', file))

def holdOut(random_seed = 42, test_size = 0.2):
    random.seed(random_seed)
    main_folder = 'datasets'

    train_images_folder = os.path.join(main_folder, 'train/images')
    train_labels_folder = os.path.join(main_folder, 'train/labels')
    val_images_folder = os.path.join(main_folder, 'val/images')
    val_labels_folder = os.path.join(main_folder, 'val/labels')

    folders = [train_images_folder, train_labels_folder, val_images_folder, val_labels_folder]

  # Sino existen los directorios
    for folder in folders:
        if not os.path.exists(folder):
            os.makedirs(folder)

    # Obtener todos los archivos de la columna 'FileName'
    all_files = metadata['FileName'].tolist()

    # Mezclar aleatoriamente los archivos
    random.shuffle(all_files,)

    split_idx = int((1-test_size) * len(all_files))
    trainFiles = all_files[:split_idx]
    valFiles = all_files[split_idx:]

    for folder in folders:
        for file in os.listdir(folder):
            if file.split(".")[0] in trainFiles:
                if file.endswith(".jpg"):
                    shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'train/images', file))
                elif file.endswith(".txt"):
                    shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'train/labels', file))
    
    for folder in folders:
        for file in os.listdir(folder):
            if file.split(".")[0] in valFiles:
                if file.endswith(".jpg"):
                    shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'val/images', file))
                elif file.endswith(".txt"):
                    shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'val/labels', file))
                    
def plot_detections(results):
    img_rgb = cv2.cvtColor(results.orig_img, cv2.COLOR_BGR2RGB)

    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img_rgb)
    for box, score in zip(results.boxes.xyxy, results.boxes.conf):
        x1, y1, x2, y2 = box
        # Coordenadas deben ser ints
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        width, height = x2 - x1, y2 - y1
        rect = patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

        # Mostrar el score como etiqueta en la esquina superior izquierda de la caja
        label = f"{score:.2f}"  # Formatear el score a 2 decimales
        ax.text(x1, y1, label, color='white', fontsize=5, verticalalignment='top', bbox={'color': 'r', 'pad': 0})
        
    plt.show()

def show_image(image_path, txt_path):
    img = cv2.imread(image_path)
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # Leer y mostrar el contenido del archivo .txt
    with open(txt_path, 'r') as file:
        annotations = file.readlines()

    for annotation in annotations:
        class_id, x_center, y_center, width, height = map(float, annotation.split())
        x = (x_center - width / 2) * img.shape[1]
        y = (y_center - height / 2) * img.shape[0]
        w = width * img.shape[1]
        h = height * img.shape[0]
        rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()

# Test

## Tipo de preprocesamiento

In [ ]:
# Elegir el tipo de preprocesamiento
holdOut(random_seed = 42, test_size = 0.2)
# onePlantOut("Nuttall oak")


## Carga de modelos

In [12]:

model8 = YOLO('Modelos\yolov8\HoldOutSimpleYolov8n.pt')
model8Adjust = YOLO('Modelos\yolov8\HoldOutAdjustYolov8n.pt')
model10 = YOLO('Modelos\yolov10\HoldOutSimpleYolov10n.pt')
model10Adjust = YOLO('Modelos\yolov10\HoldOutAdjustYolov10n.pt')

ReTrainModel10 = YOLO("Modelos\yolov10\HoldOutSimpleYolov10nRETRAIN.pt")  

## Test visual

In [ ]:
PATH_images = "datasets/train/images"
PATH_labels = "datasets/train/labels"
plant_type = '/STMHD'
number = "0002"
image_path = f"{PATH_images}{plant_type}{number}.jpg"
txt_path = f"{PATH_labels}{plant_type}{number}.txt"

results = model10.predict(image_path)[0]

show_image(image_path, txt_path)
plot_detections(results)

## Test métricas


In [14]:
results8 = model8.val(data='YOLO8INFO.yaml', save_json=False)
results8Adjust = model8Adjust.val(data='YOLO8INFO.yaml', save_json=False)
results10 = model10.val(data='YOLO8INFO.yaml', save_json=False)
results10Adjust = model10Adjust.val(data='YOLO8INFO.yaml', save_json=False)

# Contador

In [ ]:
def count_bounding_boxes(model, image_path, confidence_threshold):
    results = model.predict(image_path, verbose=False)
    bbox_counts = defaultdict(int)

    for result in results:
        for box in result.boxes: 
            if box.conf >= confidence_threshold: 
                class_id = int(box.cls) 
                bbox_counts[class_id] += 1

    return bbox_counts

def load_labels(label_path):
    labels = defaultdict(list)
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = int(parts[0])
            bbox = list(map(float, parts[1:]))
            labels[class_id].append(bbox)
    return labels

def calculate_differences(pred_counts, real_counts):
    differences = {} 

    all_classes = set(pred_counts.keys()).union(set(real_counts.keys()))

    for class_id in all_classes:
        real_count = real_counts.get(class_id, 0)
        pred_count = pred_counts.get(class_id, 0)

        difference = pred_count - real_count
        differences[class_id] = difference

    return differences

# Confidence Threshold de yolo es 0.25 por defecto
def compare_predictions_with_labels(model, image_path, label_path, confidence_threshold = 0.25): 

    pred_counts = count_bounding_boxes(model, image_path, confidence_threshold)

    real_labels = load_labels(label_path)

    real_counts = {class_id: len(bboxes) for class_id, bboxes in real_labels.items()}

    differences_by_class = calculate_differences(pred_counts, real_counts)

    return differences_by_class


In [17]:
a = count_bounding_boxes(model8,image_path,confidence_threshold=0.25)

In [ ]:
a[0]

In [ ]:
image_dir = 'datasets/val/images'
label_dir = 'datasets/val/labels'

images = [img for img in os.listdir(image_dir) if img.endswith(('.jpg', '.png'))]

differences8 = []
differences8Adjust = []
differences10 = []
differences10Adjust = []

for image_name in tqdm(images):
    image_path = os.path.join(image_dir, image_name)
    label_path = os.path.join(label_dir, image_name.replace('.jpg', '.txt').replace('.png', '.txt'))
    differences8_by_class = compare_predictions_with_labels(model8, image_path, label_path)
    differences8.append(differences8_by_class)
    differences8Adjust_by_class = compare_predictions_with_labels(model8Adjust, image_path, label_path)
    differences8Adjust.append(differences8Adjust_by_class)
    differences10_by_class = compare_predictions_with_labels(model10, image_path, label_path)
    differences10.append(differences10_by_class)
    differences10Adjust_by_class = compare_predictions_with_labels(model10Adjust, image_path, label_path)
    differences10Adjust.append(differences10Adjust_by_class)


In [33]:
def calculate_statistics(differences):
    media = np.mean(differences)
    mediana = np.median(differences)
    desviacion_estandar = np.std(differences)
    varianza = np.var(differences)
    rango = np.ptp(differences)
    minimo = np.min(differences)
    maximo = np.max(differences)
    sesgo = stats.skew(differences)
    curtosis = stats.kurtosis(differences)
    
    return {
        "media": media,
        "mediana": mediana,
        "desviacion_estandar": desviacion_estandar,
        "varianza": varianza,
        "rango": rango,
        "minimo": minimo,
        "maximo": maximo,
        "sesgo": sesgo,
        "curtosis": curtosis
    }

def flatten_differences_by_class(differences):
    class_differences = defaultdict(list)
    for diff in differences:
        for class_id, value in diff.items():
            class_differences[class_id].append(value)
    return class_differences

def calculate_and_print_stats_by_class(flat_differences):
    for class_id, diffs in flat_differences.items():
        stats_by_class = calculate_statistics(diffs)
        print(f"Clase {class_id}:")
        print(stats_by_class)
        print()


In [ ]:
flat_differences8 = flatten_differences_by_class(differences8)
flat_differences8Adjust = flatten_differences_by_class(differences8Adjust)
flat_differences10 = flatten_differences_by_class(differences10)
flat_differences10Adjust = flatten_differences_by_class(differences10Adjust)

print("Métricas estadísticas por clase para model8:")
calculate_and_print_stats_by_class(flat_differences8)

print("Métricas estadísticas por clase para model8Adjust:")
calculate_and_print_stats_by_class(flat_differences8Adjust)

print("Métricas estadísticas por clase para model10:")
calculate_and_print_stats_by_class(flat_differences10)

print("Métricas estadísticas por clase para model10Adjust:")
calculate_and_print_stats_by_class(flat_differences10Adjust)

# Test Re-Entrenados

In [ ]:
results10Retrain = ReTrainModel10.val(data='YOLO8INFO.yaml', save_json=False)

In [ ]:
image_dir = 'datasets/val/images'
label_dir = 'datasets/val/labels'

images = [img for img in os.listdir(image_dir) if img.endswith(('.jpg', '.png'))]

differences = []

for image_name in tqdm(images):
    image_path = os.path.join(image_dir, image_name)
    label_path = os.path.join(label_dir, image_name.replace('.jpg', '.txt').replace('.png', '.txt'))
    differences_by_class = compare_predictions_with_labels(ReTrainModel10, image_path, label_path)
    differences.append(differences_by_class)

In [ ]:
flat_differences = flatten_differences_by_class(differences)

print("Métricas estadísticas por clase:")
calculate_and_print_stats_by_class(flat_differences)